In [1]:
# Start up the BQL service
import bql
import pandas as pd

bq = bql.Service()   # <-- create BQL service object

In [2]:
region_country_mapping_df=pd.read_csv('region_country_mapping.csv')
master_region_list=list(region_country_mapping_df.REGION.unique())
master_region_list

['Asia',
 'AustraliaOceania',
 'EasternEurope',
 'MENA',
 'NorthAmerica',
 'SouthCentralAmerica',
 'SubsaharanAfrica',
 'WesternEurope']

In [3]:
rating_score_mapping_df=pd.read_csv('rating_score_mapping.csv')

In [4]:
# # Get Region Country Mapping
# import bql
# bq = bql.Service()

# master_region_list=['Asia','AustraliaOceania','EasternEurope','MENA','NorthAmerica',
#                     'SouthCentralAmerica','SubsaharanAfrica','WesternEurope']
# NorthAmerica = bq.univ.members(master_region_list,type='Region')
# field = bq.data.id()

# req = bql.Request(NorthAmerica,field)
# res = bq.execute(req)
# region_country_mapping_df=res[0].df()
# region_country_mapping_df.to_csv('region_country_mapping_df.csv')

In [6]:
# Get all Bonds for the ticker
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

input_ticker = 'AAPL'
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.data.ticker() == input_ticker)

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
ccy = bq.data.crncy()['value']
rating = bq.data.bb_composite()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Currency':ccy,'Rating':rating
})
res = bq.execute(req)
data = bql.combined_df(res).dropna()
data['Name']=data['Name']+' | '+data['Rating']+' | '+data['Currency']
data.head()

,Name,YTW,Time to Maturity,Currency,Rating
ID,,,,,
BK930045 Corp,AAPL 2.55 08/20/60 | AA+ | USD,4.108720,38.119097,USD,AA+
EK732419 Corp,AAPL 3.45 02/09/45 | AA+ | USD,4.314396,22.592745,USD,AA+
BQ791814 Corp,AAPL 1.7 08/05/31 | AA+ | USD,3.682113,9.075975,USD,AA+
AM383401 Corp,AAPL 3.35 02/09/27 | AA+ | USD,3.334269,4.591376,USD,AA+
JK138046 Corp,AAPL 3 ¼ 02/23/26 | AA+ | USD,3.347077,3.630390,USD,AA+


In [7]:
import bqplot as bqp
import pandas as pd
import numpy as np

# Create some random data
dataframe = data.copy()

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                      labels=['Time to Maturity', 'YTW','Name'],
                      formats=['.3f', '.3f'])
mark_scatter = bqp.Scatter(x=dataframe['Time to Maturity'],
                           y=dataframe['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y})

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

# Create Figure
figure = bqp.Figure(marks=[mark_scatter],
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title=f"Yields for {input_ticker} Bonds",
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

# Display the figure
figure


Figure(animation_duration=500, axes=[Axis(label='Time to Maturity', scale=LinearScale()), Axis(label='YTW', or…

In [26]:
# Get Region , Sector and rating for input bond
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

input_ticker = 'AAPL'
input_maturity_mm_dd_yyyy = '08/20/2060'
input_maturity_mm_dd_yyyy=pd.to_datetime(input_maturity_mm_dd_yyyy).date()
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.ticker() == input_ticker , 
                                                      bq.data.maturity() == input_maturity_mm_dd_yyyy))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
issue_date = bq.data.issue_dt()['value']
sector = bq.data.bics_level_2_industry_group_name()['value']
country = bq.data.cntry_of_risk()['value']
rating = bq.data.bb_composite()['value']
ccy = bq.data.crncy()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Issue Date':issue_date,'Sector':sector,'Country of Risk':country,
    'Rating':rating,'Currency':ccy
})
res = bq.execute(req)
data = bql.combined_df(res)
data = data.dropna().sort_values(by="Issue Date").tail(1)
data['Region']=region_country_mapping_df[region_country_mapping_df.COUNTRY_CODE==data['Country of Risk'].values[0]]['REGION'].values[0]
data['Rating Score']=rating_score_mapping_df[rating_score_mapping_df.rating==data['Rating'].values[0]]['rating_score'].values[0]
data['Name']=data['Name']+' | '+data['Rating']+' | '+data['Currency']
data.head()

,Name,YTW,Time to Maturity,Issue Date,Sector,Country of Risk,Rating,Currency,Region,Rating Score
ID,,,,,,,,,,
BK930045 Corp,AAPL 2.55 08/20/60 | AA+ | USD,4.10872,38.119097,2020-08-20,Communications Equipment,US,AA+,USD,NorthAmerica,35


In [28]:
# Get analytics for output bonds based on Region and Sector 
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

bond_sector=data['Sector'].values[0]
bond_region=data['Region'].values[0]
bond_ccy=data['Currency'].values[0]
country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.crncy()==bond_ccy, bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                      bq.data.bics_level_2_industry_group_name() == bond_sector)))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
country = bq.data.cntry_of_risk()['value']
rating=bq.data.bb_composite()['value']
ccy = bq.data.crncy()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'Currency':ccy
})
res = bq.execute(req)
datas = bql.combined_df(res)
datas = datas.dropna()
datas['Region']=bond_region
datas['Sector']=bond_sector
datas['Name']=datas['Name']+' | '+datas['Rating']+' | '+datas['Currency']
datas.head()

,Name,YTW,Time to Maturity,Country of Risk,Rating,Currency,Region,Sector
ID,,,,,,,,
ZO392848 Corp,AVYA 6 ⅛ 09/15/28 | B | USD,14.746600,6.190281,US,B,USD,NorthAmerica,Communications Equipment
BO204717 Corp,POLY 4 ¾ 03/01/29 | B | USD,4.772562,6.647502,US,B,USD,NorthAmerica,Communications Equipment
BK930045 Corp,AAPL 2.55 08/20/60 | AA+ | USD,4.108720,38.119097,US,AA+,USD,NorthAmerica,Communications Equipment
EK732419 Corp,AAPL 3.45 02/09/45 | AA+ | USD,4.314396,22.592745,US,AA+,USD,NorthAmerica,Communications Equipment
BQ791814 Corp,AAPL 1.7 08/05/31 | AA+ | USD,3.682113,9.075975,US,AA+,USD,NorthAmerica,Communications Equipment


In [ ]:
datas.Rating.unique()

In [31]:
len(datas)

114

In [29]:
import bqplot as bqp
import pandas as pd
import numpy as np

# Create some random data
dataframe = datas.copy()

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                      labels=['Time to Maturity', 'YTW','Name'],
                      formats=['.3f', '.3f'])
mark_scatter = [bqp.Scatter(x=dataframe['Time to Maturity'],
                           y=dataframe['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y}),
                bqp.Scatter(x=data['Time to Maturity'],
                           y=data['YTW'],
                           names=data['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                            colors=['#FF5A00'],
                           scales={'x': scale_x, 'y': scale_y})
               ]

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

# Create Figure
figure = bqp.Figure(marks=mark_scatter,
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title=f"Yields for {bond_sector} Bonds in {bond_region} region",
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

# Display the figure
figure


Figure(animation_duration=500, axes=[Axis(label='Time to Maturity', scale=LinearScale()), Axis(label='YTW', or…

In [33]:
# Get analytics for output bonds based on Region, Sector and Rating 
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

bond_rating_score=data['Rating Score'].values[0]
bond_region=data['Region'].values[0]
bond_sector=data['Sector'].values[0]
bond_ccy=data['Currency'].values[0]
country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
ratings_list_required=list(rating_score_mapping_df[(rating_score_mapping_df['rating_score']>=bond_rating_score-1) &
                                                   (rating_score_mapping_df['rating_score']<=bond_rating_score+1)
                                                  ]['rating'].unique())
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.crncy()==bond_ccy,
                                                      bq.func.and_(bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                      bq.data.bb_composite().in_(ratings_list_required)),
                                                     bq.data.bics_level_2_industry_group_name() == bond_sector)))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
country = bq.data.cntry_of_risk()['value']
rating=bq.data.bb_composite()['value']
ccy = bq.data.crncy()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'Currency':ccy
})
res = bq.execute(req)
datas = bql.combined_df(res)
datas = datas.dropna()
datas['Region']=bond_region
datas['Sector']=bond_sector
datas['Name']=datas['Name']+' | '+datas['Rating']+' | '+datas['Currency']
datas.head()

,Name,YTW,Time to Maturity,Country of Risk,Rating,Currency,Region,Sector
ID,,,,,,,,
BK930045 Corp,AAPL 2.55 08/20/60 | AA+ | USD,4.108720,38.119097,US,AA+,USD,NorthAmerica,Communications Equipment
EK732419 Corp,AAPL 3.45 02/09/45 | AA+ | USD,4.314396,22.592745,US,AA+,USD,NorthAmerica,Communications Equipment
BQ791814 Corp,AAPL 1.7 08/05/31 | AA+ | USD,3.682113,9.075975,US,AA+,USD,NorthAmerica,Communications Equipment
AM383401 Corp,AAPL 3.35 02/09/27 | AA+ | USD,3.334269,4.591376,US,AA+,USD,NorthAmerica,Communications Equipment
JK138046 Corp,AAPL 3 ¼ 02/23/26 | AA+ | USD,3.347077,3.630390,US,AA+,USD,NorthAmerica,Communications Equipment


In [34]:
len(datas)

50

In [35]:
import bqplot as bqp
import pandas as pd
import numpy as np

# Create some random data
dataframe = datas.copy()

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                      labels=['Time to Maturity', 'YTW','Name'],
                      formats=['.3f', '.3f'])
mark_scatter = [bqp.Scatter(x=dataframe['Time to Maturity'],
                           y=dataframe['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y}),
                bqp.Scatter(x=data['Time to Maturity'],
                           y=data['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                            colors=['#FF5A00'],
                           scales={'x': scale_x, 'y': scale_y})
               ]

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

# Create Figure
figure = bqp.Figure(marks=mark_scatter,
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title=f"Yields for {bond_sector} Bonds in {bond_region} region (Similar Rating)",
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

# Display the figure
figure


Figure(animation_duration=500, axes=[Axis(label='Time to Maturity', scale=LinearScale()), Axis(label='YTW', or…

In [36]:
#ESG Alternative
# Get Region , Sector , Currency and rating for input bond
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

input_isin = 'XS1759265264'
security = input_isin.strip()+' Corp'

bondsuniv = bq.univ.bonds(security)

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
issue_date = bq.data.issue_dt()['value']
sector = bq.data.bics_level_2_industry_group_name()['value']
country = bq.data.cntry_of_risk()['value']
rating = bq.data.bb_composite()['value']
duration = bq.data.duration()['value']
ccy = bq.data.crncy()['value']
esg_score = bq.data.esg_score(score_source='MSCI')['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Issue Date':issue_date,'Sector':sector,'Country of Risk':country,
    'Rating':rating,'Duration':duration,'Currency':ccy
})
res = bq.execute(req)
data = bql.combined_df(res)
data = data.sort_values(by="Issue Date").tail(1)
data['Region']=region_country_mapping_df[region_country_mapping_df.COUNTRY_CODE==data['Country of Risk'].values[0]]['REGION'].values[0]
data['Rating Score']=rating_score_mapping_df[rating_score_mapping_df.rating==data['Rating'].values[0]]['rating_score'].values[0]
data.head()

,Name,YTW,Time to Maturity,Issue Date,Sector,Country of Risk,Rating,Duration,Currency,Region,Rating Score
ID,,,,,,,,,,,
BQ178654 Corp,FOSUNI 3.95 10/02/26,17.680946,4.235455,2021-07-02,Travel & Lodging,CN,BB-,3.502701,EUR,Asia,24


In [1]:
input_isin = 'XS1759265264'
security = bq.univ.esgTicker(input_isin.strip()+' Corp')

esg_score = bq.data.esg_disclosure_score()['value']

req = bql.Request(security, {
    'ESG Disclosure Score':esg_score
})
res = bq.execute(req)
res[0].df()

NameError: name 'bq' is not defined

In [38]:
input_isin = 'XS1759265264'
security = bq.univ.esgTicker(input_isin.strip()+' Corp')
esg_score = bq.func.value(bq.data.esg_disclosure_score()['value'],security)


req = bql.Request(input_isin, {
    'ESG Disclosure Score':esg_score,
})
res = bq.execute(req)
res[0].df()

,ORIG_IDS,ESG Disclosure Score
ID,,
XS1759265264,656 HK Equity,73.201889


In [40]:
input_isin_raw = ['XS1759265264','US037833BA77']
input_isin = [i.strip()+' Corp' for i in input_isin_raw]
security = bq.univ.esgTicker(input_isin)
esg_score = bq.func.value(bq.data.esg_disclosure_score()['value'],security)


req = bql.Request(input_isin_raw, {
    'ESG Disclosure Score':esg_score,
})
res = bq.execute(req)
res[0].df()

,ORIG_IDS,ESG Disclosure Score
ID,,
XS1759265264,656 HK Equity,73.201889
XS1759265264,AAPL US Equity,NaN
US037833BA77,656 HK Equity,73.201889
US037833BA77,AAPL US Equity,NaN


In [ ]:
# Find all bonds for a region

import bql
bq = bql.Service()

univ = bq.univ.bondsuniv(
    'Active').filter(
bq.func.any(bq.data.cntry_of_risk()==bq.func.value(bq.data.id(),bq.univ.members('Asia', type='Region'))))

field = bq.data.id()

req = bql.Request(univ,field, with_params = dict(mode='cached'))
res = bq.execute(req)

res[0].df()

In [ ]:
p_input_bond_isin='US037833AK68'
active_bonds = bq.univ.bondsuniv('Active')
#amount_out = bq.data.amt_outstanding(Currency='USD') > 100*10**9
ticker = bq.data.ticker
filtered_bonds = bq.univ.filter(active_bonds, ticker==p_input_bond_ticker)
security = bq.data.id()

req = bql.Request(filtered_bonds, {'Security': security})
res = bq.execute(req)
data = res[0].df()
data.head()


In [29]:
aapl = bq.univ.bonds('AAPL Equity')
name = bq.data.name()
ticker = bq.data.ticker()
req = bql.Request(aapl, {'Name':name,'Ticker':ticker})
res = bq.execute(req)
data = res[0].df()
data.head(5)

,Name
ID,
BK930045 Corp,AAPL 2.55 08/20/60
EK732419 Corp,AAPL 3.45 02/09/45
BQ791814 Corp,AAPL 1.7 08/05/31
AM383401 Corp,AAPL 3.35 02/09/27
JK138046 Corp,AAPL 3 ¼ 02/23/26


In [28]:
data

,Name
ID,
BK930045 Corp,AAPL 2.55 08/20/60
EK732419 Corp,AAPL 3.45 02/09/45
BQ791814 Corp,AAPL 1.7 08/05/31
AM383401 Corp,AAPL 3.35 02/09/27
JK138046 Corp,AAPL 3 ¼ 02/23/26
...,...
EK747237 Corp,AAPL 0 ¾ 02/25/30
EK248882 Corp,AAPL 3.45 05/06/24
AN466677 Corp,AAPL 3.2 05/11/27
